# Camada Gold

## 1.0 Ingestão de dados

### 1.1 Visualizando arquivos que estão na Camada Silver

In [0]:
%fs
ls dbfs:/mnt/Formula1/Silver

### 1.2 Carregando os 3 arquivos parquet em DataFrames

In [0]:
driver_details_df = (spark.read.parquet("dbfs:/mnt/Formula1/Silver/Driver_Details.parquet/"))

race_results_df = (spark.read.parquet("dbfs:/mnt/Formula1/Silver/Race_Results.parquet/"))


##2.0 Modelagem de Dados

### 2.1 Preparando os dados para análise

In [0]:
# Dos 20 pilotos escalados para a temporada de 2025, 16 possuem histórico de
# corrida, vamos realizar o filtro:

from pyspark.sql.functions import col

pilotos_com_historico = [
    "hamilton", "max_verstappen", "alonso", "leclerc", "sainz",
    "russell", "ocon", "gasly", "norris", "bearman", "stroll",
    "hulkenberg", "lawson", "tsunoda", "piastri", "albon"
]

pilotos_temporada_2025_df = driver_details_df.filter(
    col("Piloto").isin(pilotos_com_historico)
)

In [0]:
# Junção entre os DataFrames pilotos_temporada_2025_df e race_results_df para análise

join_pilotos_corridas_df = race_results_df.join(
    pilotos_temporada_2025_df,
    on="Piloto_Id",
    how="right"
)

##3.0 Agregações e métricas

### 3.1 Indice de vitórias (aproveitamento) dos pilotos. 

### Quantidade de corridas que o piloto participou dividida pela quantidade de vezes que ficou em primeiro lugar.

In [0]:
from pyspark.sql import functions as F

# Agrupando os dados por piloto e calculando o número de corridas e vitórias
triagem_pilotos_df = join_pilotos_corridas_df.groupby("Piloto").agg(
    F.count("Resultado_Id").alias("Nº_Corridas"),
    F.sum((F.col("Ordem_Posição") == 1).cast("int")).alias("Vitorias")
)

# Calculando o índice de vitórias (%)
triagem_pilotos_df = triagem_pilotos_df.withColumn(
    "Indice_Vitorias_(%)", 
    (F.col("Vitorias") / F.col("Nº_Corridas")) * 100
)

# Arredondando o índice de vitórias para 2 casas decimais
triagem_pilotos_df = triagem_pilotos_df.withColumn(
    "Indice_Vitorias_(%)", 
    F.round(F.col("Indice_Vitorias_(%)"), 2)
)

# Ordenando os pilotos pelo índice de vitórias de forma decrescente
triagem_pilotos_df = triagem_pilotos_df.orderBy("Indice_Vitorias_(%)", ascending=False)

# Exibindo o DataFrame final
triagem_pilotos_df.show()

##4.0 Tratamento final de dados

### 4.1 Coluna indicando quando os dados foram atualizados